In [1]:
import time
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import *
from selenium import webdriver  # Selinium requires also a driver to be able to control a browser, 
                                # to be installed in the project directory from
                                #  https://selenium-python.readthedocs.io/installation.html
                                # https://github.com/mn9891/weather-scrap-selenium/blob/master/Weather_data_mtl_from_weather_gc_ca.ipynb

In [2]:
# Dictionary mapping months' numbers from pervious function with their names
month_dictio ={ 1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June',
    7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'}
inv_month_dictio = {v: k for k, v in month_dictio.items()}

def generate_dates_df(start_date,end_date,date_format='%Y-%m-%d'):
    """Generate a dataframe of the monthly spaced dates between start_date and end_date """
    """(since the available data is monthly stored on the targeted website) """
    
    s_date = datetime.datetime.strptime(start_date,date_format )
    e_date = datetime.datetime.strptime(end_date, date_format)
    counter = s_date
    dates = []
    while counter <= e_date:
        dates.append(counter.date())
        counter += relativedelta(months=1) # advance one month each iteration
    dates_list = [str(date) for date in dates]
    # creates a pandas dataframe of dates
    dates_df = pd.DataFrame(dates_list,columns=['date'])
    ff = pd.to_datetime(dates_df.date, format=date_format)
    for att in ('Day','Month','Year'):
        dates_df[att] = np.zeros(len(dates_list),int)
        for ind in ff.index:
            dates_df.loc[ind,att] = getattr(ff[ind],att.lower())
    dates_df['Month'] = dates_df['Month'].map(month_dictio)
    return dates_df

In [124]:
from selenium import webdriver # Para Mac OS
chrome_path = r'/usr/local/bin/chromedriver' #path from 'which chromedriver'
# driver = webdriver.Chrome('chromedriver_win32/chromedriver') # Para Windows OS 
driver = webdriver.Chrome(executable_path=chrome_path)
driver.get('http://climate.weather.gc.ca/climate_data/hourly_data_e.html?hlyRange=2013-02-13%7C2018-09-18&dlyRange=2013-02-14%7C2018-09-18&mlyRange=%7C&StationID=51157&Prov=QC&urlExtension=_e.html&searchType=stnName&optLimit=specDate&StartYear=2010&EndYear=2018&selRowPerPage=25&Line=0&searchMethod=contains&Month=4&Day=1&txtStationName=montreal&timeframe=1&Year=2014')
start_date = '2014-04-01'
end_date = '2018-08-01'
dates_list_df = generate_dates_df(start_date,end_date,'%Y-%m-%d')
for ind, row in dates_list_df.iterrows():
    D = row['Day']
    M = row['Month']
    Y = row['Year']
    year = driver.find_element_by_id('Year1')
    year.send_keys(Y)
    month = driver.find_element_by_id('Month1')
    month.send_keys(M)
    day = driver.find_element_by_id('Day1')
    day.send_keys(D)
    day.submit()
    download_btn = driver.find_element_by_xpath('//*[@id="climateNav"]/div[3]/section/div[1]/form/input[6]')
    download_btn.click()

In [3]:
from os import listdir
from os.path import isfile, join

In [4]:
all_csvs = [f for f in listdir('/Users/leandrorodordeolrodoriveira/Documents/github/PGR-IESB-P9041-CDNA-20203/montreal/weather_raw') if isfile(join('/Users/leandrorodordeolrodoriveira/Documents/github/PGR-IESB-P9041-CDNA-20203/montreal/weather_raw', f))]

In [5]:
all_csvs

['en_climate_hourly_QC_7025251_12-2017_P1H.csv',
 'en_climate_hourly_QC_7025251_07-2017_P1H.csv',
 'en_climate_hourly_QC_7025251_01-2016_P1H.csv',
 'en_climate_hourly_QC_7025251_06-2016_P1H.csv',
 'en_climate_hourly_QC_7025251_06-2017_P1H.csv',
 'en_climate_hourly_QC_7025251_01-2017_P1H.csv',
 'en_climate_hourly_QC_7025251_08-2018_P1H.csv',
 'en_climate_hourly_QC_7025251_12-2016_P1H.csv',
 'en_climate_hourly_QC_7025251_07-2016_P1H.csv',
 'en_climate_hourly_QC_7025251_06-2015_P1H.csv',
 'en_climate_hourly_QC_7025251_01-2015_P1H.csv',
 'en_climate_hourly_QC_7025251_02-2018_P1H.csv',
 'en_climate_hourly_QC_7025251_12-2014_P1H.csv',
 'en_climate_hourly_QC_7025251_05-2018_P1H.csv',
 'en_climate_hourly_QC_7025251_07-2014_P1H.csv',
 'en_climate_hourly_QC_7025251_12-2015_P1H.csv',
 'en_climate_hourly_QC_7025251_07-2015_P1H.csv',
 'en_climate_hourly_QC_7025251_06-2014_P1H.csv',
 'en_climate_hourly_QC_7025251_04-2018_P1H.csv',
 'en_climate_hourly_QC_7025251_03-2018_P1H.csv',
 'en_climate_hourly_

In [163]:
# Nao funciona
all_weather_data = pd.read_csv('/Users/leandrorodordeolrodoriveira/Documents/github/PGR-IESB-P9041-CDNA-20203/montreal/weather_raw/'+all_csvs[0])
for f in all_csvs:
    aux_df = pd.read_csv('/Users/leandrorodordeolrodoriveira/Documents/github/PGR-IESB-P9041-CDNA-20203/montreal/weather_raw/'+f, skiprows=15)
    all_weather_data = all_weather_data.append(aux_df,ignore_index=True)

In [18]:
import os
import glob
os.chdir("/Users/leandrorodordeolrodoriveira/Documents/github/PGR-IESB-P9041-CDNA-20203/montreal/weather_raw")

In [19]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [20]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [21]:
combined_csv

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,-73.74,45.47,MONTREAL INTL A,7025251,2017-12-01 00:00,2017,12,1,00:00,4.0,...,NaN,24.1,NaN,100.72,NaN,NaN,NaN,NaN,NaN,NaN
1,-73.74,45.47,MONTREAL INTL A,7025251,2017-12-01 01:00,2017,12,1,01:00,4.5,...,NaN,24.1,NaN,100.77,NaN,NaN,NaN,NaN,NaN,Cloudy
2,-73.74,45.47,MONTREAL INTL A,7025251,2017-12-01 02:00,2017,12,1,02:00,4.9,...,NaN,24.1,NaN,100.85,NaN,NaN,NaN,NaN,NaN,NaN
3,-73.74,45.47,MONTREAL INTL A,7025251,2017-12-01 03:00,2017,12,1,03:00,4.5,...,NaN,24.1,NaN,100.93,NaN,NaN,NaN,NaN,NaN,NaN
4,-73.74,45.47,MONTREAL INTL A,7025251,2017-12-01 04:00,2017,12,1,04:00,4.3,...,NaN,24.1,NaN,101.01,NaN,NaN,NaN,NaN,NaN,Cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,-73.74,45.47,MONTREAL INTL A,7025251,2016-11-30 19:00,2016,11,30,19:00,5.0,...,NaN,11.3,NaN,100.30,NaN,NaN,NaN,NaN,NaN,Rain
716,-73.74,45.47,MONTREAL INTL A,7025251,2016-11-30 20:00,2016,11,30,20:00,5.7,...,NaN,11.3,NaN,100.13,NaN,NaN,NaN,NaN,NaN,Rain
717,-73.74,45.47,MONTREAL INTL A,7025251,2016-11-30 21:00,2016,11,30,21:00,7.7,...,NaN,24.1,NaN,99.99,NaN,NaN,NaN,NaN,NaN,Rain
718,-73.74,45.47,MONTREAL INTL A,7025251,2016-11-30 22:00,2016,11,30,22:00,8.1,...,NaN,19.3,NaN,99.83,NaN,NaN,NaN,NaN,NaN,Rain


In [24]:
combined_csv.tail()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
715,-73.74,45.47,MONTREAL INTL A,7025251,2016-11-30 19:00,2016,11,30,19:00,5.0,...,NaN,11.3,NaN,100.30,NaN,NaN,NaN,NaN,NaN,Rain
716,-73.74,45.47,MONTREAL INTL A,7025251,2016-11-30 20:00,2016,11,30,20:00,5.7,...,NaN,11.3,NaN,100.13,NaN,NaN,NaN,NaN,NaN,Rain
717,-73.74,45.47,MONTREAL INTL A,7025251,2016-11-30 21:00,2016,11,30,21:00,7.7,...,NaN,24.1,NaN,99.99,NaN,NaN,NaN,NaN,NaN,Rain
718,-73.74,45.47,MONTREAL INTL A,7025251,2016-11-30 22:00,2016,11,30,22:00,8.1,...,NaN,19.3,NaN,99.83,NaN,NaN,NaN,NaN,NaN,Rain
719,-73.74,45.47,MONTREAL INTL A,7025251,2016-11-30 23:00,2016,11,30,23:00,7.5,...,NaN,24.1,NaN,99.71,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
combined_csv['Date/Time'] = combined_csv['Date/Time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M'))
combined_csv = combined_csv.sort_values(by=['Date/Time'])
combined_csv.tail()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
5157,-73.74,45.47,MONTREAL INTL A,7025251,2018-08-31 21:00:00,2018,8,31,21:00,21.6,...,NaN,24.1,NaN,101.90,NaN,NaN,NaN,NaN,NaN,NaN
742,-73.74,45.47,MONTREAL INTL A,7025251,2018-08-31 22:00:00,2018,8,31,22:00,21.8,...,NaN,24.1,NaN,101.89,NaN,25.0,NaN,NaN,NaN,Mostly Cloudy
5158,-73.74,45.47,MONTREAL INTL A,7025251,2018-08-31 22:00:00,2018,8,31,22:00,21.8,...,NaN,24.1,NaN,101.89,NaN,25.0,NaN,NaN,NaN,Mostly Cloudy
5159,-73.74,45.47,MONTREAL INTL A,7025251,2018-08-31 23:00:00,2018,8,31,23:00,21.1,...,NaN,24.1,NaN,101.93,NaN,26.0,NaN,NaN,NaN,NaN
743,-73.74,45.47,MONTREAL INTL A,7025251,2018-08-31 23:00:00,2018,8,31,23:00,21.1,...,NaN,24.1,NaN,101.93,NaN,26.0,NaN,NaN,NaN,NaN


In [26]:
# Change Time column format and name to be consistent with the trips data
combined_csv['Time'] = combined_csv['Date/Time'].apply(lambda x: x.hour)
combined_csv = combined_csv.rename(columns = {'Time':'Hour'})
# drop all-nan-columns
combined_csv = combined_csv.drop(columns=['Temp Flag', 'Dew Point Temp Flag','Rel Hum Flag','Wind Dir Flag', 'Wind Spd Flag', 'Wind Chill', 'Stn Press Flag', 'Visibility Flag','Hmdx Flag','Wind Chill Flag'])

In [28]:
#export all to new csv
combined_csv.to_csv('/Users/leandrorodordeolrodoriveira/Documents/github/PGR-IESB-P9041-CDNA-20203/montreal/weather_raw/treated_weather_raw.csv',index=False)

In [35]:
#export each year seperatly to new csv
for y in range(2014,2018):
    combined_csv[combined_csv.Year==y].to_csv('/Users/leandrorodordeolrodoriveira/Documents/github/PGR-IESB-P9041-CDNA-20203/montreal/weather_raw/' + str(y) + '.csv',index=False)